# A notebook for test and debug some python code

### To do
* Create a conda env [X]
*     Im having a error [X] (resolvi usando pip no ambiente)
*   ERRO TRANSFORM TORCHVISION
* Selecionar e colocar arquiteturas [X]
* unet[X]
* ddpm
* cyclogan 
* vae
* vit [X]
* Colocar funcoes de perda da tabela
* Colocar todas as metricas

### import the utils libs

In [20]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
#from torchvision import transforms 
import numpy as np
from PIL import Image



In [8]:
!pip install torchsummary torchviz torchinfo kornia[x] graphviz

In [9]:
import kornia
print(kornia.__version__)

0.7.3


In [12]:
from torchsummary import summary
import torchviz as tv
import torchinfo as tinf

In [15]:
summary?

Signature: summary(model, input_size, batch_size=-1, device='cuda')
Docstring: <no docstring>
File:      ~/anaconda3/envs/Losses/lib/python3.12/site-packages/torchsummary/torchsummary.py
Type:      function

In [14]:
tinf.summary?

Signature:
tinf.summary(
    model: 'nn.Module',
    input_size: 'INPUT_SIZE_TYPE | None' = None,
    input_data: 'INPUT_DATA_TYPE | None' = None,
    batch_dim: 'int | None' = None,
    cache_forward_pass: 'bool | None' = None,
    col_names: 'Iterable[str] | None' = None,
    col_width: 'int' = 25,
    depth: 'int' = 3,
    device: 'torch.device | str | None' = None,
    dtypes: 'list[torch.dtype] | None' = None,
    mode: 'str | None' = None,
    row_settings: 'Iterable[str] | None' = None,
    verbose: 'int | None' = None,
    **kwargs: 'Any',
) -> 'ModelStatistics'
Docstring:
Summarize the given PyTorch model. Summarized information includes:
    1) Layer names,
    2) input/output shapes,
    3) kernel shape,
    4) # of parameters,
    5) # of operations (Mult-Adds),
    6) whether layer is trainable

NOTE: If neither input_data or input_size are provided, no forward pass through the
network is performed, and the provided model information is limited to layer names.

Args:
    m

## Archtectures


### U-net

In [4]:


class DoubleConv(nn.Module):
    """
    Bloco de duas convoluções 3x3 seguidas de ReLU.
    """
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    """
    Implementação da arquitetura U-Net adaptada para geração de imagens.
    """
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        # Encoder
        self.down1 = DoubleConv(in_channels, 64)
        self.down2 = DoubleConv(64, 128)
        self.down3 = DoubleConv(128, 256)
        self.down4 = DoubleConv(256, 512)
        self.down5 = DoubleConv(512, 1024)

        # Max-pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Decoder
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv_up1 = DoubleConv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv_up2 = DoubleConv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv_up3 = DoubleConv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv_up4 = DoubleConv(128, 64)

        # Final convolution
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        d1 = self.down1(x)
        d2 = self.down2(self.pool(d1))
        d3 = self.down3(self.pool(d2))
        d4 = self.down4(self.pool(d3))
        d5 = self.down5(self.pool(d4))

        # Decoder
        u1 = self.up1(d5)
        u1 = torch.cat((u1, d4), dim=1)
        u1 = self.conv_up1(u1)
        u2 = self.up2(u1)
        u2 = torch.cat((u2, d3), dim=1)
        u2 = self.conv_up2(u2)
        u3 = self.up3(u2)
        u3 = torch.cat((u3, d2), dim=1)
        u3 = self.conv_up3(u3)
        u4 = self.up4(u3)
        u4 = torch.cat((u4, d1), dim=1)
        u4 = self.conv_up4(u4)

        return self.final_conv(u4)

if __name__ == "__main__":
    # Definindo o dispositivo (GPU se disponível)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Inicializando a U-Net
    in_channels = 3  # Por exemplo, RGB
    out_channels = 3  # Saída de imagem RGB
    model = UNet(in_channels, out_channels).to(device)

    # Exemplo de entrada
    x = torch.randn(1, in_channels, 256, 256).to(device)  # Batch size 1, 256x256 imagem
    output = model(x)

    print(x.shape, output.shape)  # Deve ser (1, out_channels, 256, 256)
     # Plotando a arquitetura da rede
    


torch.Size([1, 3, 256, 256]) torch.Size([1, 3, 256, 256])


### Torchinfo

In [5]:

model_stats = tinf.summary(model, (1, 3, 256, 256), verbose=0)
summary_str = str(model_stats)
# summary_str contains the string representation of the summary!
print(model_stats)

Layer (type:depth-idx)                   Output Shape              Param #
UNet                                     [1, 3, 256, 256]          --
├─DoubleConv: 1-1                        [1, 64, 256, 256]         --
│    └─Sequential: 2-1                   [1, 64, 256, 256]         --
│    │    └─Conv2d: 3-1                  [1, 64, 256, 256]         1,792
│    │    └─BatchNorm2d: 3-2             [1, 64, 256, 256]         128
│    │    └─ReLU: 3-3                    [1, 64, 256, 256]         --
│    │    └─Conv2d: 3-4                  [1, 64, 256, 256]         36,928
│    │    └─BatchNorm2d: 3-5             [1, 64, 256, 256]         128
│    │    └─ReLU: 3-6                    [1, 64, 256, 256]         --
├─MaxPool2d: 1-2                         [1, 64, 128, 128]         --
├─DoubleConv: 1-3                        [1, 128, 128, 128]        --
│    └─Sequential: 2-2                   [1, 128, 128, 128]        --
│    │    └─Conv2d: 3-7                  [1, 128, 128, 128]        73,856
│ 

### Torchviz

In [22]:
try:
        y = model(torch.randn(1, in_channels, 256, 256).to(device))
        tv.make_dot(y, params=dict(list(model.named_parameters()))).render("unet_architecture", format="png")
        
        #plot.format = 'png'
        
        print("Arquitetura da U-Net plotada com sucesso.")
except ImportError:
        print("Para plotar a arquitetura, instale o pacote torchviz.")


Arquitetura da U-Net plotada com sucesso.


### Torchsummary

In [11]:
summary(model, x,depth=1, dtypes=[torch.long],branching=False,verbose=1, col_width=16)

TypeError: summary() got an unexpected keyword argument 'depth'

In [6]:

class DoubleConv(nn.Module):
    """
    Bloco de duas convoluções 3x3 seguidas de ReLU.
    Permite a parametrização do uso de BatchNorm.
    """
    def __init__(self, in_channels, out_channels, use_batch_norm=True):
        super(DoubleConv, self).__init__()
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        ]
        if use_batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))

        layers.extend([
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        ])
        if use_batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        
        self.double_conv = nn.Sequential(*layers)

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    """
    Implementação da arquitetura U-Net parametrizável para geração de imagens.
    Permite a parametrização do número de blocos de convolução e o uso de BatchNorm.
    """
    def __init__(self, in_channels, out_channels, base_filters=64, num_layers=5, use_batch_norm=True):
        super(UNet, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.base_filters = base_filters
        self.num_layers = num_layers
        self.use_batch_norm = use_batch_norm

        # Encoder
        self.encoder_layers = nn.ModuleList()
        filters = base_filters
        
        for i in range(num_layers-1):
            self.encoder_layers.append(DoubleConv(in_channels, filters, use_batch_norm))
            in_channels = filters
            filters *= 2
        
        # Max-pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bottleneck
        self.bottleneck = DoubleConv(filters // 2, filters, use_batch_norm)

        # Decoder
        self.up_layers = nn.ModuleList()
        self.up_convs = nn.ModuleList()
        
        
        for i in range(num_layers-1, 0, -1):#range(num_layers-1, 0, -1)
            filters //= 2
            self.up_layers.append(nn.ConvTranspose2d(filters * 2, filters, kernel_size=2, stride=2))
            self.up_convs.append(DoubleConv(filters * 2, filters , use_batch_norm))
            

        # Final convolution
        
        self.final_conv = nn.Conv2d(base_filters, out_channels, kernel_size=1)
    def see_model(self):
        print(f"""
        encoder: {self.encoder_layers}
        pool: {self.pool}
        botleneck: {self.bottleneck}
        up laeyers: {self.up_layers}
        up_convs: {self.up_convs}
        final conv: {self.final_conv}""")

    def forward(self, x):
        # Encoder
        encoder_results = []
        for layer in self.encoder_layers:
            x = layer(x)
            encoder_results.append(x)
            x = self.pool(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Decoder
        for i, (up, conv) in enumerate(zip(self.up_layers, self.up_convs)):
            x = up(x)
            x = torch.cat((x, encoder_results[-(i+1)]), dim=1)
            x = conv(x)

        return self.final_conv(x)

def standardize_image(image):
    """
    Função para padronizar as imagens de entrada.
    """
    mean = image.mean()
    std = image.std()
    return (image - mean) / std

if __name__ == "__main__":
    # Definindo o dispositivo (GPU se disponível)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Inicializando a U-Net parametrizável
    in_channels = 3  # Por exemplo, RGB
    out_channels = 3  # Saída de imagem RGB
    base_filters = 64
    num_layers = 4
    use_batch_norm = True
    model = UNet(in_channels, out_channels, base_filters, num_layers, use_batch_norm).to(device)
    #model.see_model()

    # Exibindo a arquitetura da rede
    #summary(model, input_size=(in_channels, 256, 256))

    # Exemplo de entrada
    x = torch.randn(1, in_channels, 256, 256).to(device)  # Batch size 1, 256x256 imagem
    # x = standardize_image(x)
    output = model(x)

    print(output.shape)  # Deve ser (1, out_channels, 256, 256)


torch.Size([1, 3, 256, 256])


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [ ]:

# Plotando a arquitetura da rede
try:

    y = model(torch.randn(1, in_channels, 256, 256).to(device))
    tv.make_dot(y, params=dict(list(model.named_parameters()))).render("unet_architecture_2", format="png")
    print("Arquitetura da U-Net plotada com sucesso.")
except ImportError:
    print("Para plotar a arquitetura, instale o pacote torchviz.")


In [50]:
summary(model, x,depth=1, dtypes=[torch.long],branching=False,verbose=1, col_width=16)

Layer (type:depth-idx)                   Output Shape     Param #
DoubleConv: 2-1                          [-1, 64, 256, 256] --
MaxPool2d: 1-1                           [-1, 64, 128, 128] --
DoubleConv: 2-2                          [-1, 128, 128, 128] --
MaxPool2d: 1-2                           [-1, 128, 64, 64] --
DoubleConv: 2-3                          [-1, 256, 64, 64] --
MaxPool2d: 1-3                           [-1, 256, 32, 32] --
DoubleConv: 1-4                          [-1, 512, 32, 32] --
Sequential: 2-4                          [-1, 512, 32, 32] --
ConvTranspose2d: 2-5                     [-1, 256, 64, 64] 524,544
DoubleConv: 2-6                          [-1, 256, 64, 64] --
ConvTranspose2d: 2-7                     [-1, 128, 128, 128] 131,200
DoubleConv: 2-8                          [-1, 128, 128, 128] --
ConvTranspose2d: 2-9                     [-1, 64, 256, 256] 32,832
DoubleConv: 2-10                         [-1, 64, 256, 256] --
Conv2d: 1-5                              [

Layer (type:depth-idx)                   Output Shape     Param #
DoubleConv: 2-1                          [-1, 64, 256, 256] --
MaxPool2d: 1-1                           [-1, 64, 128, 128] --
DoubleConv: 2-2                          [-1, 128, 128, 128] --
MaxPool2d: 1-2                           [-1, 128, 64, 64] --
DoubleConv: 2-3                          [-1, 256, 64, 64] --
MaxPool2d: 1-3                           [-1, 256, 32, 32] --
DoubleConv: 1-4                          [-1, 512, 32, 32] --
Sequential: 2-4                          [-1, 512, 32, 32] --
ConvTranspose2d: 2-5                     [-1, 256, 64, 64] 524,544
DoubleConv: 2-6                          [-1, 256, 64, 64] --
ConvTranspose2d: 2-7                     [-1, 128, 128, 128] 131,200
DoubleConv: 2-8                          [-1, 128, 128, 128] --
ConvTranspose2d: 2-9                     [-1, 64, 256, 256] 32,832
DoubleConv: 2-10                         [-1, 64, 256, 256] --
Conv2d: 1-5                              [

### Limpa memoria CUDA

In [13]:
import gc

# Verifique a memória alocada e reservada
print(f'Memória alocada: {torch.cuda.memory_allocated()} bytes')
print(f'Memória reservada: {torch.cuda.memory_reserved()} bytes')

# Force a coleta de lixo para liberar a memória imediatamente
gc.collect()

# Libere o cache de memória do PyTorch
torch.cuda.empty_cache()

# Verifique a memória alocada e reservada
print(f'Memória alocada: {torch.cuda.memory_allocated()} bytes')
print(f'Memória reservada: {torch.cuda.memory_reserved()} bytes')

print(f"Memoria limpa")

Memória alocada: 0 bytes
Memória reservada: 0 bytes
Memória alocada: 0 bytes
Memória reservada: 0 bytes
Memoria limpa


### DDPM

### CycleGAN


In [26]:
import os
if not os.path.isdir("pytorch-CycleGAN-and-pix2pix"):
    !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
os.chdir("pytorch-CycleGAN-and-pix2pix")
!pip install -r requirements.txt 
!ls

CycleGAN.ipynb	docs		 LICENSE  pix2pix.ipynb     scripts   util
data		environment.yml  models   README.md	    test.py
datasets	imgs		 options  requirements.txt  train.py


In [ ]:
#Falta aprender a resgatar o modelo

### VAE

In [34]:
os.chdir("..")
!ls

loss	    pytorch-CycleGAN-and-pix2pix  unet_architecture_2.png
Losses.yml  README.md			  unet_architecture.png
main.py     src				  vit_architecture.png
metrics     unet_architecture		  work.ipynb
models	    unet_architecture_2


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

# Encoder parametrizável com BatchNorm
class Encoder(nn.Module):
    def __init__(self, input_channels, hidden_dims, latent_dim):
        super(Encoder, self).__init__()
        
        # Inicializa uma lista para armazenar as camadas da rede
        modules = []
        
        # Adiciona camadas de convolução e batch normalization conforme especificado em hidden_dims
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(input_channels, h_dim, kernel_size=4, stride=2, padding=1),
                    nn.BatchNorm2d(h_dim),
                    nn.ReLU()
                )
            )
            input_channels = h_dim
        
        # Define as camadas finais para calcular mu e logvar
        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1] * 16 * 16, latent_dim)  # Supondo entrada de 256x256
        self.fc_logvar = nn.Linear(hidden_dims[-1] * 16 * 16, latent_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

# Decoder parametrizável com BatchNorm
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dims, output_channels):
        super(Decoder, self).__init__()
        
        # Inicializa uma lista para armazenar as camadas da rede
        modules = []
        
        # Primeira camada completamente conectada para expandir a representação latente
        self.fc = nn.Linear(latent_dim, hidden_dims[-1] * 16 * 16)
        
        # Adiciona camadas de convolução transposta e batch normalization conforme especificado em hidden_dims
        hidden_dims.reverse()
        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=4,
                                       stride=2,
                                       padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.ReLU()
                )
            )
        
        # Camada final para reconstrução da imagem
        modules.append(
            nn.Sequential(
                nn.ConvTranspose2d(hidden_dims[-1],
                                   output_channels,
                                   kernel_size=4,
                                   stride=2,
                                   padding=1),
                nn.Sigmoid()
            )
        )
        
        self.decoder = nn.Sequential(*modules)
    
    def forward(self, z):
        x = torch.relu(self.fc(z))
        x = x.view(x.size(0), -1, 16, 16)
        x = self.decoder(x)
        return x

# Definição do VAE parametrizável
class VAE(nn.Module):
    def __init__(self, input_channels, hidden_dims, latent_dim, output_channels):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_channels, hidden_dims, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dims, output_channels)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Configuração de parâmetros
latent_dim = 16
hidden_dims = [32, 64, 128, 256]
input_channels = 3
output_channels = 3

# Inicialização do modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(input_channels=input_channels, hidden_dims=hidden_dims, latent_dim=latent_dim, output_channels=output_channels).to(device)

# Teste com tensor randômico
model.eval()
with torch.no_grad():
    # Gera um tensor randômico com formato (1, 3, 256, 256)
    random_tensor = torch.randn(1, 3, 256, 256).to(device)
    
    # Passa o tensor randômico pelo encoder para obter mu e logvar
    mu, logvar = model.encoder(random_tensor)
    
    # Reamostragem para obter o vetor latente z
    z = model.reparameterize(mu, logvar)
    
    # Passa o vetor latente pelo decoder para gerar a imagem reconstruída
    generated_image, _, _ = model(random_tensor)
    
    print("Generated Image Shape:", generated_image.shape)  # Esperado: (1, 3, 256, 256)


Generated Image Shape: torch.Size([1, 3, 256, 256])


In [19]:
tinf.summary(model, (1,3,256,256),depth=4)

Layer (type:depth-idx)                        Output Shape              Param #
VAE                                           [1, 3, 256, 256]          --
├─Encoder: 1-1                                [1, 16]                   --
│    └─Sequential: 2-1                        [1, 256, 16, 16]          --
│    │    └─Sequential: 3-1                   [1, 32, 128, 128]         --
│    │    │    └─Conv2d: 4-1                  [1, 32, 128, 128]         1,568
│    │    │    └─BatchNorm2d: 4-2             [1, 32, 128, 128]         64
│    │    │    └─ReLU: 4-3                    [1, 32, 128, 128]         --
│    │    └─Sequential: 3-2                   [1, 64, 64, 64]           --
│    │    │    └─Conv2d: 4-4                  [1, 64, 64, 64]           32,832
│    │    │    └─BatchNorm2d: 4-5             [1, 64, 64, 64]           128
│    │    │    └─ReLU: 4-6                    [1, 64, 64, 64]           --
│    │    └─Sequential: 3-3                   [1, 128, 32, 32]          --
│    │    │ 

### Vit

In [21]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim)
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.attn(x, x, x)
        x = x + self.dropout(attn_output)
        x = self.norm1(x)
        mlp_output = self.mlp(x)
        x = x + self.dropout(mlp_output)
        x = self.norm2(x)
        return x


In [22]:
class ImageEnhancerTransformer(nn.Module):
    def __init__(self, image_size=256, patch_size=16, num_channels=3, embed_dim=512, num_heads=8, mlp_dim=1024, num_layers=6, dropout=0.1):
        super(ImageEnhancerTransformer, self).__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2
        self.patch_dim = num_channels * patch_size * patch_size
        self.embed_dim = embed_dim

        self.patch_embeddings = nn.Linear(self.patch_dim, embed_dim)
        self.position_embeddings = nn.Parameter(torch.zeros(1, self.num_patches, embed_dim))

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_dim, dropout) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, self.patch_dim)

    def forward(self, x):
        batch_size = x.size(0)
        x = self._to_patches(x)
        x = self.patch_embeddings(x) + self.position_embeddings

        for block in self.transformer_blocks:
            x = block(x)

        x = self.norm(x)
        x = self.fc(x)
        x = self._from_patches(x, batch_size)
        return x

    def _to_patches(self, x):
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, channels, self.image_size // self.patch_size, self.patch_size,
                   self.image_size // self.patch_size, self.patch_size)
        x = x.permute(0, 2, 4, 3, 5, 1).contiguous()
        x = x.view(batch_size, self.num_patches, -1)
        return x

    def _from_patches(self, x, batch_size):
        x = x.view(batch_size, self.image_size // self.patch_size, self.image_size // self.patch_size,
                   self.patch_size, self.patch_size, -1)
        x = x.permute(0, 5, 1, 3, 2, 4).contiguous()
        x = x.view(batch_size, -1, self.image_size, self.image_size)
        return x


In [23]:
class DummyDataset(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

    def __len__(self):
        return 100  # número de exemplos fictício

    def __getitem__(self, idx):
        # Gera uma imagem aleatória como entrada e uma versão "melhorada" como alvo
        input_image = Image.fromarray((np.random.rand(256, 256, 3) * 255).astype(np.uint8))
        target_image = Image.fromarray((np.random.rand(256, 256, 3) * 255).astype(np.uint8))

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

# transform = transforms.Compose([
#     transforms.ToTensor()
# ])

dataset = DummyDataset()
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

model = ImageEnhancerTransformer()
# Gera um tensor aleatório como entrada
input_tensor = torch.rand(1, 3, 256, 256)  # Batch size de 1, 3 canais, 256x256 de resolução

# Passa o tensor pelo modelo
output_tensor = model(input_tensor)

# Mostra as formas dos tensores de entrada e saída
print("Forma do tensor de entrada:", input_tensor.shape)
print("Forma do tensor de saída:", output_tensor.shape)

tinf.summary(model, (1,3, 256, 256),depth=3)
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# # Loop de treinamento
# for epoch in range(10):  # número de épocas fictício
#     for inputs, targets in dataloader:
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")


Forma do tensor de entrada: torch.Size([1, 3, 256, 256])
Forma do tensor de saída: torch.Size([1, 3, 256, 256])


Layer (type:depth-idx)                   Output Shape              Param #
ImageEnhancerTransformer                 [1, 3, 256, 256]          131,072
├─Linear: 1-1                            [1, 256, 512]             393,728
├─ModuleList: 1-2                        --                        --
│    └─TransformerBlock: 2-1             [1, 256, 512]             --
│    │    └─MultiheadAttention: 3-1      [1, 256, 512]             1,050,624
│    │    └─Dropout: 3-2                 [1, 256, 512]             --
│    │    └─LayerNorm: 3-3               [1, 256, 512]             1,024
│    │    └─Sequential: 3-4              [1, 256, 512]             1,050,112
│    │    └─Dropout: 3-5                 [1, 256, 512]             --
│    │    └─LayerNorm: 3-6               [1, 256, 512]             1,024
│    └─TransformerBlock: 2-2             [1, 256, 512]             --
│    │    └─MultiheadAttention: 3-7      [1, 256, 512]             1,050,624
│    │    └─Dropout: 3-8                 [1, 256

In [ ]:
# Plotando a arquitetura da rede
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
try:

    y = model(torch.randn(1, 3, 256, 256).to(device))
    tv.make_dot(y, params=dict(list(model.named_parameters()))).render("vit_architecture", format="png")
    print("Arquitetura da U-Net plotada com sucesso.")
except ImportError:
    print("Para plotar a arquitetura, instale o pacote torchviz.")


Arquitetura da U-Net plotada com sucesso.


In [24]:
### Limpa memoria CUDA
import gc

# Verifique a memória alocada e reservada
print(f'Memória alocada: {torch.cuda.memory_allocated()} bytes')
print(f'Memória reservada: {torch.cuda.memory_reserved()} bytes')

# Force a coleta de lixo para liberar a memória imediatamente
gc.collect()

# Libere o cache de memória do PyTorch
torch.cuda.empty_cache()

# Verifique a memória alocada e reservada
print(f'Memória alocada: {torch.cuda.memory_allocated()} bytes')
print(f'Memória reservada: {torch.cuda.memory_reserved()} bytes')

print(f"Memoria limpa")

Memória alocada: 0 bytes
Memória reservada: 0 bytes
Memória alocada: 0 bytes
Memória reservada: 0 bytes
Memoria limpa


## Losses

In [ ]:
import torch
import torch.nn as nn

class GANLoss(nn.Module):
    def __init__(self, use_lsgan=True):
        super(GANLoss, self).__init__()
        self.use_lsgan = use_lsgan
        if use_lsgan:
            self.criterion = nn.MSELoss()
        else:
            self.criterion = nn.BCELoss()

    def forward(self, output, target_is_real):
        target = torch.ones_like(output) if target_is_real else torch.zeros_like(output)
        loss = self.criterion(output, target)
        return loss


### Color loss	 	 


In [ ]:
class ColorLoss(nn.Module):
    def __init__(self):
        super(ColorLoss, self).__init__()
        self.criterion = nn.MSELoss()

    def forward(self, input, target):
        return self.criterion(input, target)



### SSIM PSNR MS-SSIM	 	 


In [33]:
from kornia.losses import ssim_loss as ssim, psnr_loss as psnr, MS_SSIMLoss as ms_ssim, charbonnier_loss as charbonnier
import torch
import numpy as np

from torch import nn
input1 = torch.rand(1, 3, 255, 255)
input2 = torch.rand(1, 3, 255, 255)
# print(np.max(input1.numpy()), np.max(input2.numpy()))

loss = ssim(input1, input2, 11)

loss2 = psnr(input1,input2, np.max(input1.numpy())) # 10 * log(4/((1.2-1)**2)) / log(10)
loss3 = ms_ssim()

loss4 = charbonnier(input1, input2,reduction='mean')

# class SSIMLoss(nn.Module):
#     def __init__(self):
#         super(SSIMLoss, self).__init__()

#     def forward(self, img1, img2):
#         return ssim(img1, img2)
# class PSNRLoss(nn.Module):
#     def __init__(self):
#         super(PSNRLoss, self).__init__()

#     def forward(self, img1, img2):
#         return (img1, img2)
print(loss.item(), loss2.item(), loss3(input1, input2).item(), loss4.item())


0.49616897106170654 -7.792870998382568 66.98175811767578 0.07644809782505035


### MAE	 		 	 	 


In [ ]:
class MAELoss(nn.Module):
    def __init__(self):
        super(MAELoss, self).__init__()
        self.criterion = nn.L1Loss()

    def forward(self, input, target):
        return self.criterion(input, target)


### MSE	 	 	 	 	 


In [ ]:
class MSELoss(nn.Module):
    def __init__(self):
        super(MAELoss, self).__init__()
        self.criterion = nn.MSELoss()

    def forward(self, input, target):
        return self.criterion(input, target)


### Uranker	 	 	 	 	 


### MUSIQ	 	 	 	 	 


### Content Loss VGG	 	 	 	 	 


### Acumlative Superiority	 	 	 	 	 


### Discriminative	 	 	 	 	 


### L2	 	 	 	 	 


### L1	 	 	 	 	 


### CIoU	 	 	 	 	 


### Adversarial Loss	 	 	 	 	 


### WGan Loss	 	 	 	 	 


### Gan loss	 	 


ModuleNotFoundError: No module named 'kornia'

### Gradient Loss	 	 	 	 	 


### Color Structure Perceptual	 	 	 	 	 


### Detail loss	 	 	 	 	 


### CGan	 	 	 	 	 


### Angular	 	 	 	 	 


### margin-ranking loss	 	 	 	 	 


### Classifier loss	 	 	 	 	 


### Identity mapping	 	 	 	 	 


### Cycle consistency loss	 	 	 	 	 


### LAB loss	 	 	 	 	 


### LCH loss

## Metrics